<a href="https://colab.research.google.com/github/runnithan03/Dissertation/blob/main/Dissertation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lasso Regression

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

print("Generating sample dataset...")

# Generate a sample dataset
X_train, y_train = make_classification(n_samples=100, n_features=20, n_informative=2, n_redundant=10, random_state=42)

# Print the shapes of X_train and y_train to verify
print("Shapes of X_train and y_train:")
print(X_train.shape)  # Should output (100, 20)
print(y_train.shape)  # Should output (100,)

print("Fitting Lasso (L1) Regularization model...")

# Lasso (L1) Regularization
model_lasso = LogisticRegression(penalty='l1', solver='saga', max_iter=10000).fit(X_train, y_train)
print("Lasso model coefficients:", model_lasso.coef_)
print("Lasso model accuracy on training data:", accuracy_score(y_train, model_lasso.predict(X_train)))

print("Fitting Ridge (L2) Regularization model...")

# Ridge (L2) Regularization
model_ridge = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=10000).fit(X_train, y_train)
print("Ridge model coefficients:", model_ridge.coef_)
print("Ridge model accuracy on training data:", accuracy_score(y_train, model_ridge.predict(X_train)))

print("Script completed.")



Generating sample dataset...
Shapes of X_train and y_train:
(100, 20)
(100,)
Fitting Lasso (L1) Regularization model...
Lasso model coefficients: [[ 0.          0.          0.          0.          0.28613678  0.
  -2.11185444  0.          0.          0.          0.          0.
   0.          0.          0.          0.20170223  0.          3.20555137
   0.          0.        ]]
Lasso model accuracy on training data: 0.99
Fitting Ridge (L2) Regularization model...
Ridge model coefficients: [[ 0.53796691  0.22762154 -0.56878911 -0.19871216  0.56092924  0.12798941
  -1.06670917 -0.07043501  0.15573116 -0.11013136 -0.75531367  0.32347209
  -0.83402903 -0.33845405 -0.03755675  0.41659081  0.20915689  0.97093934
   0.67589751 -0.84130427]]
Ridge model accuracy on training data: 1.0
Script completed.


Pytorch

In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 1. Basic Tensor Operations
# Create a tensor
x = torch.tensor([[1, 2], [3, 4]], dtype=torch.float)
print(f"Tensor x:\n{x}")

# Create a tensor filled with zeros
zeros = torch.zeros((2, 3))
print(f"Zeros tensor:\n{zeros}")

# Create a random tensor
random_tensor = torch.rand((2, 2))
print(f"Random tensor:\n{random_tensor}")

# Element-wise addition
y = torch.tensor([[5, 6], [7, 8]], dtype=torch.float)
result_add = x + y
print(f"Element-wise addition result:\n{result_add}")

# Matrix multiplication
result_mul = torch.matmul(x, y)
print(f"Matrix multiplication result:\n{result_mul}")

# Move tensor to GPU (if available)
x_gpu = x.to(device)
print(f"Tensor x on {device}:\n{x_gpu}")

# 2. Define a Simple Neural Network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(784, 128)  # Fully connected layer 1
        self.fc2 = nn.Linear(128, 64)   # Fully connected layer 2
        self.fc3 = nn.Linear(64, 10)    # Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate the model and move it to the appropriate device
model = SimpleNN().to(device)

# 3. Prepare Data - Using the MNIST Dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load the training and test datasets
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# 4. Set Up Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 5. Training the Neural Network
epochs = 5

for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in trainloader:
        # Flatten the input images
        inputs = inputs.view(inputs.shape[0], -1).to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}")

print("Training complete.")

# 6. Evaluate the Model
correct = 0
total = 0

with torch.no_grad():  # No need to calculate gradients during evaluation
    for inputs, labels in testloader:
        inputs = inputs.view(inputs.shape[0], -1).to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

# 7. Save the Model
torch.save(model.state_dict(), 'model.pth')
print("Model saved to 'model.pth'.")

# 8. Load the Model
#loaded_model = SimpleNN()
#loaded_model.load_state_dict(torch.load('model.pth'))
#loaded_model = loaded_model.to(device)
#print("Model loaded and ready for inference.")


Using device: cpu
Tensor x:
tensor([[1., 2.],
        [3., 4.]])
Zeros tensor:
tensor([[0., 0., 0.],
        [0., 0., 0.]])
Random tensor:
tensor([[0.3011, 0.3242],
        [0.1040, 0.5545]])
Element-wise addition result:
tensor([[ 6.,  8.],
        [10., 12.]])
Matrix multiplication result:
tensor([[19., 22.],
        [43., 50.]])
Tensor x on cpu:
tensor([[1., 2.],
        [3., 4.]])
Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 15739736.50it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 488639.53it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4367422.03it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2448024.77it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch 1, Loss: 0.41992934463994464
Epoch 2, Loss: 0.17773972845463548
Epoch 3, Loss: 0.12797877673051758
Epoch 4, Loss: 0.10175308126555101
Epoch 5, Loss: 0.08649959495571106
Training complete.
Accuracy on the test set: 96.91%
Model saved to 'model.pth'.
Model loaded and ready for inference.


<ipython-input-2-d1fe061ac674>:119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load('model.pth'))
